In [64]:
#importing the requests library
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
import yfinance as yf #using the yahoo finance library 
import yahoo_fin.stock_info as si #using the yahoo_fin library 
import warnings
warnings.filterwarnings('ignore')

Getting stock data using the open source yahoo finance library. More information can be found [here](https://pypi.org/project/yfinance/)

In [2]:
#function to get stock data of a company

def stock_info(ticker):
    #define dates for 5 years
    start_date = '2017-01-01'
    end_date = '2022-12-31'
    df = yf.download(ticker,start=start_date,end=end_date)
    
    return df

In [3]:
#function to calculate stock return

def stock_return(ticker):
    df = stock_info(ticker) # run the function to get the stock data
    df_return = df[['Adj Close']]
    df_return['pct_change'] = df_return['Adj Close'].pct_change() # use pct change to calculate the diff 
    df_return.drop(df_return.index[0],inplace=True) # drop the Nan value from the first row
    
    #make a plot of the return
    fig,ax = plt.subplots(figsize=(15,8))
    ax.plot(df_return['pct_change']);
    ax.set_title('Stock Return Pct from 2017 to 2022:' +' ' + ticker);
    plt.show()
    
    return fig,ax;
      

In [4]:
def calculate_beta(ticker):
    symbols = [ticker,'SPY']
    data = yf.download(symbols, start = '2019-01-01',end = '2022-12-31')['Adj Close']
    price_change = data.pct_change()
    price_change.drop(price_change.index[0],inplace=True)
    
    X = np.array(price_change[ticker]).reshape((-1,1))
    y = np.array(price_change['SPY'])

    X_train,X_test,y_train,y_test = train_test_split(X,y)
    lr = LinearRegression()
    lr.fit(X_train,y_train)

    y_preds = lr.predict(X_test)

    fig,ax = plt.subplots(figsize=(12,7));
    ax.plot(X_test,y_preds,linestyle=':',color='orange');
    ax.scatter(y_test,y_preds,alpha=0.5);
    ax.set_title(f' Beta value = {lr.coef_}');
    plt.show()
    
    return fig,ax

## stock evaluation

2. p/e ratio
3. p/b ratio
4. div yield